# Setting up inputs for the gridded DBPM run
**Author**: Denisse Fierro Arcos  
**Date**: 2025-01-10  
  
In this notebook, we will prepare all necessary inputs need to run the gridded version of DBPM. Note that this step needs to be completed only once for each model run.  

## Loading relevant libraries

In [1]:
import os
os.chdir('/g/data/vf71/la6889/dbpm_southern_ocean/scripts/')
from glob import glob
import xarray as xr
import json
import pandas as pd
import numpy as np
import useful_functions as uf
from dask.distributed import Client

## Start a cluster for parallelisation

In [2]:
client = Client(threads_per_worker = 1)

## Defining output folder

In [2]:
base_folder = '/g/data/vf71/la6889/dbpm_inputs/east_antarctica/'
model_res = '1deg'
reg_int = 'fao-58'
out_folder = os.path.join(base_folder, 'gridded_params', model_res)
os.makedirs(out_folder, exist_ok = True)

Setting variable names to save outputs.

In [3]:
#Get the correct model resolution in arcmin to add to file name
if model_res == '1deg':
    model_res_arc = '60arcmin'
elif model_res == '025deg':
    model_res_arc = '15arcmin'

## Transforming DBPM parameters to Python-friendly format
These parameters are the outputs of the `sizeparam` function for R. They were calculated and stored in the script ([04_calculating_dbpm_fishing_params](new_workflow/04_calculating_dbpm_fishing_params.R)) as a `json` file.

In [4]:
#Loading gridded parameters
gridded_params = json.load(open(
    os.path.join(out_folder, f'dbpm_gridded_size_params_{reg_int}.json')))

#Transforming them to Python friendly format
gridded_python = uf.gridded_param_python(gridded_params)

### Adding useful entries to gridded parameters
These variables are used multiple times within the `gridded_size_model` script, so they will be added for convenience.

In [5]:
# Size classes
log10_size_bins = np.array(gridded_python['log10_size_bins'])
# Minimum (log10) size class for predators
log10_ind_min_pred_size = log10_size_bins[gridded_python['ind_min_pred_size']]
ind_min_detritivore_size = gridded_python['ind_min_detritivore_size']
log10_ind_min_detritivore_size = log10_size_bins[ind_min_detritivore_size]
log10_ind_min_fish_pred = log10_size_bins[gridded_python['ind_min_fish_pred']]
log10_ind_min_fish_det = log10_size_bins[gridded_python['ind_min_fish_det']]

#Adding new variables to gridded parameters
gridded_python['log10_ind_min_pred_size'] = log10_ind_min_pred_size
gridded_python['log10_ind_min_detritivore_size'] = log10_ind_min_detritivore_size

### Saving gridded parameters

In [186]:
out_gridded = os.path.join(out_folder, f'dbpm_gridded_size_params_{reg_int}_python.json')

#Save to disk
with open(out_gridded, 'w') as outfile:
    json.dump(gridded_python, outfile)

## Loading gridded input data
Gridded data are needed both as direct input to DBPM and to calculate params, such as habitat preference, pelagic predator and benthic detritivore size spectrum, among other things.

In [6]:
depth = xr.open_zarr(glob(
    os.path.join(base_folder, 'gridded', model_res, 
                 f'gfdl-mom6-cobalt2_obsclim_deptho_*_{reg_int}_fixed.zarr/'))[0])['deptho']

In [8]:
#Loading intercept data for spinup period
int_phy_zoo_spinup = xr.open_zarr(glob(
    os.path.join(base_folder, 'gridded', model_res,
                 f'gfdl-mom6-cobalt2_spinup_intercept_*_{reg_int}_monthly_1841_1960.zarr/'))[0])['intercept']

#Loading intercept data for model period and fixing time values
int_phy_zoo  = xr.open_zarr(glob(
    os.path.join(base_folder, 'gridded', model_res,
                 f'gfdl-mom6-cobalt2_obsclim_intercept_*_{reg_int}_monthly_1961_2010.zarr/'))[0])['intercept']

In [9]:
#Loading sea surface temperature data for spinup period
sea_surf_temp_spinup = xr.open_zarr(glob(
    os.path.join(base_folder, 'gridded', model_res,
                 f'gfdl-mom6-cobalt2_spinup_tos_*_{reg_int}_monthly_1841_1960.zarr'))[0])['tos']

#Loading sea surface temperature data for model period and fixing time values
sea_surf_temp  = xr.open_zarr(glob(
    os.path.join(base_folder, 'gridded', model_res,
                 f'gfdl-mom6-cobalt2_obsclim_tos_*_{reg_int}_monthly_1961_2010.zarr/'))[0])['tos']

In [10]:
#Loading bottom ocean temperature data for spinup period
sea_floor_temp_spinup = xr.open_zarr(glob(
    os.path.join(base_folder, 'gridded', model_res,
                 f'gfdl-mom6-cobalt2_spinup_tob_*_{reg_int}_monthly_1841_1960.zarr'))[0])['tob']

#Loading bottom ocean temperature data for model period and fixing time values
sea_floor_temp  = xr.open_zarr(glob(
    os.path.join(base_folder, 'gridded', model_res,
                 f'gfdl-mom6-cobalt2_obsclim_tob_*_{reg_int}_monthly_1961_2010.zarr/'))[0])['tob']

### Creating time variable that contains a timestep for initialisation of DBPM
The initialisation time is the month prior to the earliest timestep in the `spinup` period. If a `spinup` period is not necessary, then this will be the month prior to the start of the model.

In [191]:
time_init = [pd.Timestamp(int_phy_zoo_spinup.time[0].values)-pd.DateOffset(months = 1)]

## Creating gridded variables needed for DBPM run
We will create data array versions of the gridded inputs.

### DBPM size classes

In [11]:
# Log10 size of individuals found in the model
log10_size_bins_mat = xr.DataArray(data = log10_size_bins,
                                   dims = ['size_class'], 
                                   coords = {'size_class': log10_size_bins})

The `log10_size_bins_mat` only needs to be saved once per because size classes are the same for a set of experiments. This variable does **not** vary across space or time.

In [149]:
log10_size_bins_mat.name = 'size_bins'
log10_size_bins_mat.to_zarr('outputs/log10_size_bins_matrix.zarr/',
                            consolidated = True, mode = 'w')

### Fishing effort
Effort data for the `spinup` and modelling period is provided in a single entry in `gridded_python`. We will split this data into the `spinup` and modelling period.

In [193]:
effort_spinup = (xr.DataArray(gridded_python['effort'][:len(int_phy_zoo_spinup.time)], 
                              dims = 'time', 
                              coords = {'time': int_phy_zoo_spinup.time}).
                 expand_dims({'lat': depth.lat, 'lon': depth.lon}).
                 transpose('time', 'lat', 'lon')).where(np.isfinite(depth))

effort = (xr.DataArray(gridded_python['effort'][len(int_phy_zoo_spinup.time):],
                       dims = 'time', coords = {'time': int_phy_zoo.time}).
          expand_dims({'lat': depth.lat, 'lon': depth.lon}).
          transpose('time', 'lat', 'lon')).where(np.isfinite(depth))

In [194]:
effort_spinup.name = 'effort'
effort_out = effort_spinup.chunk({'lon': len(effort_spinup.lon), 
                                  'lat': len(effort_spinup.lat), 'time': 120})
effort_out.to_zarr(
    os.path.join(out_folder, f'effort_spinup_{model_res_arc}_{reg_int}_monthly_1841_1960.zarr/'),
                   consolidated = True, mode = 'w')

effort.name = 'effort'
effort_out = effort.chunk({'lon': len(effort.lon), 'lat': len(effort.lat), 'time': 120})
effort_out.to_zarr(
    os.path.join(out_folder, f'effort_{model_res_arc}_{reg_int}_monthly_1961_2010.zarr/'),
                   consolidated = True, mode = 'w')

### Setting habitat preferences
Predator coupling to benthic prey, which is depth dependent, 0.75 above 500 m, 0.5 between 500 and 1800 m, and 0 below 1800 m. Predator coupling to pelagic prey is equal to `1-(benthic coupling)`.  
  
These values were suggested by Clive Trueman based on stable isotope work, and proportion of biomass. Rockall Trough studies.

In [195]:
pref_benthos = (0.8*np.exp((-1/250*depth)))
pref_pelagic = (1-pref_benthos)

In [196]:
pref_benthos.name = 'pref_benthos'
pref_benthos.to_zarr(os.path.join(out_folder, 
                                  f'pref-benthos_{model_res_arc}_{reg_int}_fixed.zarr/'),
                     consolidated = True, mode = 'w')
pref_pelagic.name = 'pref_pelagic'
pref_pelagic.to_zarr(os.path.join(out_folder, 
                                  f'pref-pelagic_{model_res_arc}_{reg_int}_fixed.zarr/'), 
                     consolidated = True, mode = 'w')

### Building metabolic requirements lookup table 
Metabolic requirements are estimated per size class

In [197]:
met_req_log10_size_bins = uf.expax_f(log10_size_bins_mat, 
                                     gridded_python['metabolic_req_pred'])

In [198]:
consume_pelagic = (pref_pelagic*gridded_python['hr_volume_search']*
                   met_req_log10_size_bins)
consume_benthos = (pref_benthos*gridded_python['hr_volume_search']*
                   met_req_log10_size_bins)

In [199]:
consume_pelagic.name = 'consume_pelagic'
consume_pelagic.to_zarr(os.path.join(out_folder,
                                     f'consume-pelagic_{model_res_arc}_{reg_int}_fixed.zarr/'),
                        consolidated = True, mode = 'w')

consume_benthos.name = 'consume_benthos'
consume_benthos.to_zarr(os.path.join(out_folder,
                                     f'consume-benthos_{model_res_arc}_{reg_int}_fixed.zarr/'),
                        consolidated = True, mode = 'w')

### Calculating constant growth and mortality

In [200]:
pred_prey_mat = uf.pred_prey_matrix(log10_size_bins)

constant_growth = xr.DataArray(uf.gphi_f(pred_prey_mat, 
                                         gridded_python['log10_pred_prey_ratio'],
                                         gridded_python['log_prey_pref']),
                               dims = ['size_class', 'sc'])

constant_mortality = xr.DataArray(uf.mphi_f(-pred_prey_mat, 
                                            gridded_python['log10_pred_prey_ratio'],
                                            gridded_python['log_prey_pref'],
                                            gridded_python['metabolic_req_pred']),
                                  dims = ['size_class', 'sc'])

In [201]:
constant_growth.name = 'constant_growth'
constant_growth.to_zarr(os.path.join(out_folder,
                                     f'const-growth_{model_res_arc}_{reg_int}_fixed.zarr/'),
                        consolidated = True, mode = 'w')

constant_mortality.name = 'constant_mortality'
constant_mortality.to_zarr(os.path.join(out_folder,
                                        f'const-mort_{model_res_arc}_{reg_int}_fixed.zarr/'),
                        consolidated = True, mode = 'w')

### Creating a gridded time series of intercept of plankton size spectrum 
The size spectrum was estimated by the `GetPPIntSlope` function using GFDL-MOM6-COBALT2 outputs in script ([01_processing_dbpm_global_inputs](new_workflow/01_processing_dbpm_global_inputs.ipynb)).

In [13]:
ui0_spinup = 10**int_phy_zoo_spinup
ui0 = 10**int_phy_zoo

In [14]:
ui0_spinup.name = 'ui0'
ui0_spinup.to_zarr(os.path.join(out_folder,
                                f'ui0_spinup_{model_res_arc}_{reg_int}_monthly_1841_1960.zarr/'),
                   consolidated = True, mode = 'w')

ui0.name = 'ui0'
ui0.to_zarr(os.path.join(out_folder,
                         f'ui0_{model_res_arc}_{reg_int}_monthly_1961_2010.zarr/'),
            consolidated = True, mode = 'w')

### Initiliasing values for pelagic predators

In [204]:
predators = (xr.DataArray(data = gridded_python['init_pred'], 
                          dims = ['size_class'], 
                          coords = {'size_class': log10_size_bins}).
             expand_dims({'time': time_init, 'lat': depth.lat, 'lon': depth.lon}))

#Applying spatial mask
predators = predators.where(np.isfinite(depth))

In [205]:
predators.name = 'predators'
predators.to_zarr(os.path.join(out_folder, 
                               f'predators_{model_res_arc}_{reg_int}_init_1840.zarr/'),
                  consolidated = True, mode = 'w')

### Detritivore biomass and detritus
Creating gridded versions of `detritivores` and `detritus` initalising values to be used in gridded DBPM.

In [206]:
init_detritivores = (xr.DataArray(data = gridded_python['init_detritivores'], 
                                  dims = ['size_class'], 
                                  coords = {'size_class': log10_size_bins}).
                     expand_dims({'lat': depth.lat, 'lon': depth.lon}))

# Assigning initial values (time = 0) for detritivores from gridded parameters
detritivores = xr.where((init_detritivores.size_class >= log10_ind_min_detritivore_size),
                        init_detritivores, 0)
# Adding time dimension
detritivores = detritivores.expand_dims({'time': time_init})

#Applying spatial mask
detritivores = detritivores.where(np.isfinite(depth))

In [207]:
detritivores.name = 'detritivores'
detritivores.to_zarr(os.path.join(out_folder, 
                                  f'detritivores_{model_res_arc}_{reg_int}_init_1840.zarr/'),
                     consolidated = True, mode = 'w')

In [208]:
# Initialising detritus
detritus = xr.zeros_like(detritivores.isel(size_class = 0).drop_vars('size_class'))
# Assigning initial values (time = 0) for detritus from gridded parameters
detritus = xr.where(detritus == 0, gridded_python['init_detritus'], detritus)
#Applying spatial mask
detritus = detritus.where(np.isfinite(depth))

In [209]:
detritus.name = 'detritus'
detritus.to_zarr(os.path.join(out_folder, 
                              f'detritus_{model_res_arc}_{reg_int}_init_1840.zarr/'),
                 consolidated = True, mode = 'w')

### Note
If no initial values for `predators` and `detritivores` are included in the gridded parameters, use the following code instead:  
  
```
predators = xr.where(predators.size_class >= log10_size_bins[120],
                     0, predators)

detritivores = xr.where(detritivores.size_class >= log10_size_bins[120], 
                        0, detritivores)
```

### Other intrinsic natural mortality
The same intrinsic natural mortality values apply for `detritivores` and `predators`. It is calculated only once, but this variable is saved separately for each group. This is done because the DBPM gridded model needs mortality separately for each group.

In [210]:
other_mort = gridded_python['natural_mort']*10**(-0.25*log10_size_bins_mat)

In [211]:
other_mort.name = 'other_mort_pred'
other_mort.to_zarr(os.path.join(out_folder,
                                f'other-mort-pred_{model_res_arc}_{reg_int}_fixed.zarr/'),
                   consolidated = True, mode = 'w')

other_mort.name = 'other_mort_det'
other_mort.to_zarr(os.path.join(out_folder,
                                f'other-mort-det_{model_res_arc}_{reg_int}_fixed.zarr/'),
                   consolidated = True, mode = 'w')


### Senescence mortality
Senescence mortality rate limits large fish from building up in the system. Same function as in Law et al 2008. Using chosen parameters below gives similar M2 values as in Hall et al. 2006.  
  
As above, this variable is the same for `detritivores` and `predators`, but saved separately because DBPM model needs a mortality due to senescence for each group.

In [212]:
senes_mort = (gridded_params['const_senescence_mort']*
                      10**(gridded_params['exp_senescence_mort']*
                           (log10_size_bins_mat-
                            gridded_params['size_senescence'])))

In [213]:
senes_mort.name = 'senes_mort_pred'
senes_mort.to_zarr(os.path.join(out_folder,
                                f'senes-mort-pred_{model_res_arc}_{reg_int}_fixed.zarr/'),
                   consolidated = True, mode = 'w')

senes_mort.name = 'senes_mort_det'
senes_mort.to_zarr(os.path.join(out_folder,
                                f'senes-mort-det_{model_res_arc}_{reg_int}_fixed.zarr/'),
                   consolidated = True, mode = 'w')

### Fishing mortality
Here `fishing_mort_pred` and `fishing_mort_det` are fixed catchability terms for `predators` and `detritivores`. Fishing mortality is estimated only for individuals within fished sizes (`ind_min_fish_pred` or `ind_min_fish_det`).

In [214]:
# Using fish_mort_pred as a base for fishing mortality for predators
fish_mort_pred = xr.DataArray(
    np.repeat(gridded_python['fish_mort_pred'], 
              np.prod(consume_pelagic.shape)).reshape(consume_pelagic.shape),
    dims = ['lat', 'lon', 'size_class'],
    coords = {'lat': consume_pelagic.lat, 'lon': consume_pelagic.lon,
             'size_class': consume_pelagic.size_class})

# Changing mortality to zero outside the predator size fished
fishing_mort_pred = xr.where((fish_mort_pred.size_class >= log10_ind_min_fish_pred) &
                          (fish_mort_pred.size_class < 
                           fish_mort_pred.size_class.max()),
                          fish_mort_pred, 0).where(np.isfinite(depth))

In [215]:
fishing_mort_pred.name = 'fish_mort_pred'
fishing_mort_pred.to_zarr(os.path.join(out_folder, 
                                       f'fish-mort-pred_{model_res_arc}_{reg_int}_fixed.zarr/'),
                          consolidated = True, mode = 'w')

In [216]:
# Using fish_mort_det as a base for fishing mortality for detritivores
fish_mort_det = xr.DataArray(
    np.repeat(gridded_python['fish_mort_detritivore'], 
              np.prod(consume_benthos.shape)).reshape(consume_benthos.shape),
    dims = ['lat', 'lon', 'size_class'],
    coords = {'lat': consume_benthos.lat, 'lon': consume_benthos.lon,
             'size_class': consume_benthos.size_class})

# Changing mortality to zero outside the detritivore size fished
fishing_mort_det = xr.where((fish_mort_det.size_class >= log10_ind_min_fish_det) &
                          (fish_mort_det.size_class < 
                           fish_mort_det.size_class.max()),
                          fish_mort_det, 0).where(np.isfinite(depth))

In [217]:
fishing_mort_det.name = 'fish_mort_det'
fishing_mort_det.to_zarr(os.path.join(
    out_folder, f'fish-mort-det_{model_res_arc}_{reg_int}_fixed.zarr/'),
                         consolidated = True, mode = 'w')

### Temperature effect on pelagic organisms
This will be calculated for the `spinup` and modelling periods

In [15]:
pel_tempeffect_spinup = np.exp(gridded_python['c1']-gridded_python['activation_energy']/
                               (gridded_python['boltzmann']*(sea_surf_temp_spinup+273)))

pel_tempeffect = np.exp(gridded_python['c1']-gridded_python['activation_energy']/
                        (gridded_python['boltzmann']*(sea_surf_temp+273)))

In [16]:
pel_tempeffect_spinup.name = 'pel_temp_eff' 
pel_tempeffect_out = pel_tempeffect_spinup.chunk({'lat': 106, 'lon': 480})
pel_tempeffect_out.to_zarr(
    os.path.join(out_folder, 
                 f'pel-temp-eff_spinup_{model_res_arc}_{reg_int}_monthly_1841_1960.zarr/'),
    consolidated = True, mode = 'w')

pel_tempeffect.name = 'pel_temp_eff' 
pel_tempeffect_out = pel_tempeffect.chunk({'lat': 106, 'lon': 480})
pel_tempeffect_out.to_zarr(
    os.path.join(out_folder, f'pel-temp-eff_{model_res_arc}_{reg_int}_monthly_1961_2010.zarr/'),
    consolidated = True, mode = 'w')

### Temperature effect on benthic organisms
This will be calculated for the `spinup` and modelling periods

In [17]:
ben_tempeffect_spinup = np.exp(gridded_python['c1']-gridded_python['activation_energy']/
                               (gridded_python['boltzmann']*(sea_floor_temp_spinup+273)))

ben_tempeffect = np.exp(gridded_python['c1']-gridded_python['activation_energy']/
                        (gridded_python['boltzmann']*(sea_floor_temp+273)))

In [18]:
ben_tempeffect_spinup.name = 'ben_temp_eff'
ben_tempeffect_out = ben_tempeffect_spinup.chunk({'lat': 106, 'lon': 480})
ben_tempeffect_out.to_zarr(
    os.path.join(out_folder, 
                 f'ben-temp-eff_spinup_{model_res_arc}_{reg_int}_monthly_1841_1960.zarr/'),
    consolidated = True, mode = 'w')

ben_tempeffect.name = 'ben_temp_eff'
ben_tempeffect_out = ben_tempeffect.chunk({'lat': 106, 'lon': 480})
ben_tempeffect_out.to_zarr(
    os.path.join(out_folder, f'ben-temp-eff_{model_res_arc}_{reg_int}_monthly_1961_2010.zarr/'),
    consolidated = True, mode = 'w')

### Note
If `temp_effect` is not needed, then simply assign `1` to `pel_tempeffect` and `ben_tempeffect`:  
  
```
pel_tempeffect = 1
ben_tempeffect = 1
```